<a href="https://colab.research.google.com/github/pamslover/pamellaPfe/blob/main/WebScrapingJD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 4.7 MB/s eta 0:00:00


In [141]:
!pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941188 sha256=c33ed78228e8d599cd33566c4cdac2cb2ced7e0f21f9a6afcdc9c8d553e8de13
  Stored in directory: /root/.cache/pip/wheels/c5/01/a4/0160c55074707b535a6757a541842817d530d8080ca943a107
Successfully built langid


In [2]:
import glob
import unicodedata
from bs4 import BeautifulSoup as bs
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.core.display import display, HTML

In [3]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:

html_contents_emploiscm = []
html_contents_jobinfos = []
for file_name in sorted(glob.glob('/content/drive/MyDrive/projet pfe/emploi.cm/*.html')):
    with open(file_name, 'r') as f:
        html_contents_emploiscm.append(f.read())
for file_name in sorted(glob.glob('/content/drive/MyDrive/projet pfe/jobinfocamer.com/*.html')):
    with open(file_name, 'r') as f:
        html_contents_jobinfos.append(f.read())

In [19]:
print(f"nous avons charger {len(html_contents_emploiscm)} HTML files de emplois.cm")
print(f"nous avons charger {len(html_contents_jobinfos)} HTML files de job info")

nous avons charger 185 HTML files de emplois.cm
nous avons charger 2223 HTML files de job info


In [20]:
soup_objects_emploicm = []
for html in html_contents_emploiscm:
    soup = bs(html)
    assert soup.title is not None
    assert soup.body is not None
    soup_objects_emploicm.append(soup)

In [21]:
soup_objects_jobinfo = []
for html in html_contents_jobinfos:
    soup = bs(html)
    assert soup.title is not None
    assert soup.body is not None
    soup_objects_jobinfo.append(soup)

In [22]:
html_dict_emploicm = {'Title': [], 'Body': []}
html_dict_jobinfo = {'Title': [], 'Body': []}

In [23]:
for soup in soup_objects_emploicm :
    divs = soup.findAll('div', class_="jobs-ad-details block")
    title = divs[0].find('span', attrs={'class':'ad-ss-title'})
    title = str(title.text)
    title = title.split(":")
    title.pop(0)
    title = ''.join(title)
    body = divs[0].findAll('div', class_='content clearfix')
    html_dict_emploicm['Title'].append(title)
    html_dict_emploicm['Body'].append(body)

In [24]:
for soup in soup_objects_jobinfo :
    title = soup.find('div', class_='headline')
    title = title.find('h2').text.strip()
    body = soup.findAll('div', class_='job-description')
    html_dict_jobinfo['Title'].append(title)
    html_dict_jobinfo['Body'].append(body)

In [25]:
df_jobs = pd.DataFrame(html_dict_jobinfo)
df_emploi = pd.DataFrame(html_dict_emploicm)


In [26]:
assert len(set(html_contents_emploiscm )) == len(html_contents_emploiscm)

In [27]:
assert len(set(html_contents_jobinfos)) == len(html_contents_jobinfos)

In [28]:
df_jobs['jobs'] =[[' '.join (bullet.text.strip()
                      for bullet in soup[0].find_all('li'))]
                      for soup in html_dict_jobinfo['Body']]

In [29]:
df_emploi['jobs'] = [[' '.join (bullet.text.strip()
                      for bullet in soup[0].find_all('li'))]
                      for soup in html_dict_emploicm['Body']]

In [32]:
bulleted_post_count = 0
for bullet_list in df_emploi.jobs:
    if bullet_list:
        bulleted_post_count += 1
 
percent_bulleted = 100 * bulleted_post_count / df_emploi.shape[0]
print(f"{percent_bulleted:.2f}% of the postings contain jobs")


100.00% of the postings contain jobs


In [165]:
 df = df_emploi.append(df_jobs, ignore_index=True)

In [130]:
def remplace(text):
  text = " ".join(text)
  return text.replace("’"," ")

In [131]:
df['jobs'] = df.jobs.apply(lambda x: remplace(text = x))

In [133]:
import unidecode
import re

class cleaner:

    def __init__(self) -> None:
        pass
    
   
    def clean_text(sentence):
        # sentence = " ".join(sentence)
        text=unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf8') 
        text =cleaner.remove_special_char(cleaner.extract_only_text(text))
        # text = cleaner.extract_only_text(text)
        return cleaner.remove_double_space(text)
    
    def remove_double_space(text):
         return re.sub(r'([\s])\1{1,}|([\.])\1{1,}', r'\1', str(text))
        #  return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8')

    def extract_only_text(text):
        text= text.lower()
        text= re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', ' ', str(text))
        text= re.sub(r'[^ ]+\.[^ ]+', ' ', str(text))
        text= re.sub(r"’", " ",str(text))
        # text = re.sub('\S*www\S*\s?', ' ', str(text))
        text = re.sub('http\S+\s*', ' ', str(text))
        # text = re.sub('(\d+\.)+\d*.*','',str(text))
        text = re.sub('#S+', ' ', str(text))  # remove hashtags
        text = re.sub('@S+', '  ', str(text))  # remove mentions
        text = re.sub('\s+', ' ', str(text))  # remove extra whitespace
        text = re.sub("\t",' ',str(text))
        text = re.sub('\n|\r',' ',str(text))
        text = re.sub('[^\x00-\x7f]',' ',str(text))
        text= re.sub(r'\x00|\uf001','fi',str(text))
        text = re.sub(r'@+[A-Za-z0-9]+', ' ', str(text)) 
        text= re.sub(r'\x0c',' ',str(text))
        text= re.sub(r'www[a-z0-9\.\-+_]+\.[a-z]+[/-][a-zA-Z]?[a-z\-]+', r' ', str(text))
        text = text.replace('\\xa0', ' ')  
        # text= re.sub(r'\uf005|\uf006|\uf10b|\ue81e|\ue811|\ue80d|\ue81a|\ue803', r' ', str(text))
      
        return text
    
    def remove_special_char(text:str):
        return re.sub(r"[^a-zA-Z0-9\-“”\s\'\.]","", str(text))

In [134]:
# df["jobs"] = df['jobs'].asty
df['jobs'] = df.jobs.apply(lambda x: cleaner.clean_text(sentence = x))

In [136]:
data = df.drop(columns="Body")

data.to_csv('JobDescription.csv') 

In [137]:
Df = pd.read_csv('JobDescription.csv', engine='python' ) 
Df= Df.drop(columns='Unnamed: 0')

In [162]:
def rank_words(text_list,language):
    my_stop_words = text.ENGLISH_STOP_WORDS.union([language])
    vectorizer = TfidfVectorizer(stop_words=my_stop_words)
    tfidf_matrix = vectorizer.fit_transform(text_list).toarray()
    data = pd.DataFrame({'Words': vectorizer.get_feature_names(),
                       'Summed TFIDF': tfidf_matrix.sum(axis=0)})
    sorted_df = data.sort_values('Summed TFIDF', ascending=False)
    return sorted_df
 


In [156]:
all_bullets = []
for bullet_list in df.jobs:
    all_bullets.extend(bullet_list)
 
#sorted_df = rank_words(all_bullets, "french")
#print(sorted_df[:5].to_string(index=False))
print(len(all_bullets))

3846617


In [167]:
all_bullets = []
for bullet_list in df.jobs:
    all_bullets.extend(bullet_list)
 
sorted_df = rank_words(all_bullets, "french")
print(sorted_df[:5].to_string(index=False))
# print(len(all_bullets))

Words  Summed TFIDF
   de    404.048858
   et    326.604339
  des    265.257640
  les    193.759226
   la    188.401870


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [168]:
sorted_df

,Words,Summed TFIDF
5107,de,404.048858
7069,et,326.604339
5278,des,265.257640
10427,les,193.759226
10279,la,188.401870
...,...,...
5587,discernment,0.010019
5600,discontent,0.010019
10050,jds,0.010019
14871,readjustment,0.010019
